# Background

We aim to compute the following quantity:
$$
\mathbb{P}(\Omega^{\text{vis}}) 
  = \sum_{k=2}^{n} \mathbb{P}(\Omega^{\text{vis}}_k)
$$
whereby
\begin{equation}
\mathbb{P}(\Omega^{\text{vis}}_k) = p^{\neg \rho}_{n,k} \frac{\rho}{k-1+\rho} q_{k+1,(0,0)}
\end{equation}

In order to do this, we need to compute a series of **auxiliary variables**:
\begin{equation}
p^{\neg \rho}_{n_1,n_2} = \prod_{n_2 < k \leq n_1} \frac{k-1}{k-1+\rho}
\end{equation}
The probabilities $(q_{k,{i,j}})_{k \in \mathbb{N}, i,j \in \lbrace 0,1,2 \rbrace}$ satisfy the following recursive linear system:
\begin{align}
q_{k,(0,0)} &= \frac{(k-2) ( q_{k-1,(1,0)} + q_{k-1,(0,1)}) + \binom{k-2}{2} q_{k-1,(0,0)} }{\binom{k}{2} + \frac{k}{2} \rho} \\
%q_{k,(0,0)} &= \frac{1}{\binom{k}{2} + \frac{k}{2} \rho} \left( (k-2) ( q_{k-1,(1,0)} + q_{k-1,(0,1)}) + \binom{k-2}{2} q_{k-1,(0,0)} \right) \\
q_{k,(0,1)} &= \frac{\frac{1}{2} \theta_2 \, q_{k,(0,2)} + (k-2) \, q_{k-1,(1,1)} + \binom{k-1}{2} \, q_{k-1,(0,1)} }{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_2} \\
q_{k,(1,0)} &= \frac{\frac{1}{2} \theta_1 \, q_{k,(2,0)} + (k-2) \, q_{k-1,(1,1)} + \binom{k-1}{2} \, q_{k-1,(1,0)} }{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_1} \\
q_{k,(0,2)} &= \frac{(k-2) q_{k-1,(1,2)} + \binom{k-1}{2} q_{k-1,(0,2)} }{\binom{k}{2} + \frac{k}{2} \rho} \\
q_{k,(2,0)} &= \frac{(k-2) q_{k-1,(2,1)} + \binom{k-1}{2} q_{k-1,(2,1)} }{\binom{k}{2} + \frac{k}{2} \rho} \\q_{k,(1,1)} &= \frac{\frac{1}{2} \theta_1 \, q_{k,(2,1)} + \frac{1}{2} \theta_2 q_{k,(1,2)} + \left( \binom{k}{2} - 1 \right) \, q_{k-1,(1,1)} }{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2} ( \theta_1 + \theta_2 ) } \\
q_{k,(1,2)} &= \frac{\frac{1}{2} \theta_1 q_{k,(2,2)} + \binom{k-1}{2} q_{k-1,(1,2)}}{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_1}\\
q_{k,(2,1)} &= \frac{\frac{1}{2} \theta_2 q_{k,(2,2)} + \binom{k-1}{2} q_{k-1,(2,1)}}{\binom{k}{2} + \frac{k}{2} \rho + \frac{1}{2}\theta_2}
\end{align}
with boundary conditions:
\begin{align}
q_{k,(0,0)} &= 0 \text{ if } k<4 \\
q_{k,(0,1)} &= 0 \text{ if } k<3 \\
q_{k,(1,0)} &= 0 \text{ if } k<3 \\
q_{k,(0,2)} &= 0 \text{ if } k<3 \\
q_{k,(2,0)} &= 0 \text{ if } k<3 \\
q_{k,(1,1)} &= 0 \text{ if } k<2 \\
q_{k,(1,2)} &= 0 \text{ if } k<2 \\
q_{k,(2,1)} &= 0 \text{ if } k<2 \\
q_{k,(2,2)} &= p^{\neg \rho}_{k,1} = \prod_{i = 2}^k \frac{i-1}{i-1+\rho}
\end{align}

The solution of this system will be a series of rational functions of integers $n \geq k > 1$ and non-negative reals $\rho, \theta_1, \theta_2$.

In [57]:
def p_neg(n1,n2,rho):
    'The probability that a set of n1 lineages will coalesce to n2 lineages before experiencing recurison'
    assert n1 >= n2
    assert n2 > 0
    assert rho >= 0
    return reduce( lambda x,y: x*y, [float(k - 1)/float(k-1+rho) for k in range(n1,n2,-1)], 1.0)

In [58]:
def q_recursive(k,i,j,theta1,theta2,rho):
    'compute q_{k,(i,j)} using tail-recursion; theta1,theta2, and rho should be non-negative reals.'
    
    if   (i,j) == (0,0):
        if k < 4:
            return float(0)
        else:
            a = q_recursive(k-1, 1, 0, theta1,theta2,rho)
            b = q_recursive(k-1, 0, 1, theta1,theta2,rho)
            c = q_recursive(k-1, 0, 0, theta1,theta2,rho)
            result = ( (k - 2)*(a+b) + choose2(k-2)*c ) / float(chose2(k) + 0.5*k*rho)
            return result
            
    elif (i,j) == (0,1):
        if k < 3:
            return float(0)
        else:
            a = q_recursive(k  , 0, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 1, 1, theta1,theta2,rho)
            c = q_recursive(k-1, 0, 1, theta1,theta2,rho)
            result = ( 0.5*theta2*a + (k - 2)*b + choose2(k-1)*c ) / float(chose2(k) + 0.5*(k*rho + theta2))
            return result

    elif (i,j) == (1,0):
        if k < 3:
            return float(0)
        else:
            a = q_recursive(k  , 2, 0, theta1,theta2,rho)
            b = q_recursive(k-1, 1, 1, theta1,theta2,rho)
            c = q_recursive(k-1, 1, 0, theta1,theta2,rho)
            result = ( 0.5*theta1*a + (k - 2)*b + choose2(k-1)*c ) / float(chose2(k) + 0.5*(k*rho + theta1))
            return result

    elif (i,j) == (0,2):
        if k < 3:
            return float(0)
        else:
            a = q_recursive(k-1, 1, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 0, 2, theta1,theta2,rho)
            result = ( (k - 2)*a + choose2(k-1)*b ) / float(chose2(k) + 0.5*k*rho)
            return result
    
    elif (i,j) == (2,0):
        if k < 3:
            return float(0)
        else:
            a = q_recursive(k-1, 2, 1, theta1,theta2,rho)
            b = q_recursive(k-1, 2, 0, theta1,theta2,rho)
            result = ( (k - 2)*a + choose2(k-1)*b ) / float(chose2(k) + 0.5*k*rho)
            return result
    
    elif (i,j) == (1,1):
        if k < 2:
            return float(0)
        else:
            a = q_recursive(k  , 2, 1, theta1,theta2,rho)
            b = q_recursive(k  , 1, 2, theta1,theta2,rho)
            c = q_recursive(k-1, 1, 1, theta1,theta2,rho)
            result = ( 0.5*theta1*a + 0.5*theta2*b + (choose2(k) - 1)*c ) / float(chose2(k) + 0.5*(k*rho + theta1 + theta2))
            return result
    
    elif (i,j) == (1,2):
        if k < 2:
            return float(0)
        else:
            a = q_recursive(k  , 2, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 1, 2, theta1,theta2,rho)
            result = ( 0.5*theta1*a + choose2(k-1)*b ) / float(chose2(k) + 0.5*(k*rho + theta1))
            return result
    
    elif (i,j) == (2,1):
        if k < 2:
            return float(0)
        else:
            a = q_recursive(k  , 2, 2, theta1,theta2,rho)
            b = q_recursive(k-1, 2, 1, theta1,theta2,rho)
            result = ( 0.5*theta2*a + choose2(k-1)*b ) / float(chose2(k) + 0.5*(k*rho + theta2))
            return result
    
    elif (i,j) == (2,2):
        result = p_neg(k,1,rho)
        return result
    
    else:
        raise ValueError('0 <= i,j <= 2 must hold; arguments here: i = %i, j=%i'%(i,j))

def choose2(k):
    return k*(k-1)/2

In [59]:
q_recursive(50,0,0,1e10,1e10,0.0)

0.933605437401415

## Towards conditionals

We now define the following:

\begin{align}
\mathbb{P} (\Omega_k)
   &=  \frac{\rho}{k-1 + \rho} \frac{k}{k + \rho} \prod_{i=2}^n \frac{i-1}{i - 1 + \rho} \\
   &=  \frac{\rho}{k-1 + \rho} \frac{k}{k + \rho} p^{\neg \rho}_{n,1}
\end{align}
and
\begin{align}
\mathbb{P}(\Omega_{2,\ldots,n})
   &=  \sum_{k = 2}^n \mathbb{P} (\Omega_k) \\
   &=  p^{\neg \rho}_{n,1} \sum_{k = 2}^n \frac{\rho}{k-1 + \rho} \frac{k}{k + \rho}
\end{align}

In [67]:
def P_omega_k(k,n,rho):
    return p_neg(n,1,rho) * ( (rho*k) / ( (k-1+rho) * (k+rho) ) )

def P_omega_2toN(n,rho):
    return p_neg(n,1,rho) * sum([(rho*k) / ( (k-1+rho) * (k+rho) ) for k in xrange(2,n)])

In [68]:
P_omega_k(2,2,1.0)

0.16666666666666666